In [ ]:
from azmeta.access import reservations, resource_id
import azmeta.notebook.interactive as azmi
import pandas as pd
from datetime import datetime,timezone,timedelta
azmi.connect_kqlmagic()

In [ ]:
%%kql
let last5bills = (Usage | distinct BillingPeriodStartDate | top 5 by BillingPeriodStartDate);
Usage
| where BillingPeriodStartDate in (last5bills)
| where AccountName == 'DevOps Teams (652030-10500339)' 
    and ((ConsumedService == 'Microsoft.Compute' and MeterCategory in ('Storage', 'Virtual Machines'))
    or (ConsumedService == 'Microsoft.Storage' and MeterSubCategory contains 'Page Blob'))
| extend Type = strcat(MeterCategory, '-', case(isempty(ReservationId), 'PAYG', 'Reserved'))
| summarize sum(Cost) by BillingPeriodStartDate, Type
| evaluate pivot(Type, sum(sum_Cost))
| extend BillingPeriodStartDate = format_datetime(BillingPeriodStartDate, 'yyyy-MM')
| sort by BillingPeriodStartDate asc
| render columnchart with (ytitle="$", xtitle="Billing Period", title="IaaS Cost Reserved vs Unreserved: Past 5 Bills")

In [ ]:
%%kql -pl={"barmode":"stack"}
let last5bills = Usage 
    | distinct BillingPeriodStartDate 
    | top 10 by BillingPeriodStartDate;
let unused = Usage
    | where BillingPeriodStartDate in (last5bills)
    | where ChargeType == 'UnusedReservation';
let top4locs = unused
    | summarize cost=sum(Cost) by ResourceLocation
    | top 4 by cost
    | project ResourceLocation;
unused
| summarize sum(Cost) by location=iif(ResourceLocation in (top4locs), tolower(ResourceLocation), '_other'), BillingPeriodStartDate
| extend BillingPeriodStartDate = format_datetime(BillingPeriodStartDate, 'yyyy-MM')
| order by BillingPeriodStartDate asc, location desc
| render columnchart with (kind=stacked,xcolumn=BillingPeriodStartDate,title='Unused Reservation Cost by Region: Past 10 Bills',xtitle='Billing Period', ytitle='$')

In [ ]:
%%kql -pl={"barmode":"stack"}
let last5Bills = (Usage | distinct BillingPeriodStartDate | top 5 by BillingPeriodStartDate);
let inScopeReservations = materialize(Usage
| where BillingPeriodStartDate in (last5Bills)
| where ChargeType == "UnusedReservation"
| summarize cost=sum(Cost) by ReservationId
| top 10 by cost
| project ReservationId);
let topOffenders = materialize(Usage
| where BillingPeriodStartDate in (last5Bills)
| where ReservationId in (inScopeReservations)
| summarize cost=sum(Cost) by ReservationName, ChargeType
| order by cost desc);
topOffenders
| where ChargeType == 'Usage'
| join kind=leftouter (topOffenders | where ChargeType == 'UnusedReservation' | project-rename sortCost=cost) on ReservationName
| union (topOffenders | where ChargeType == 'UnusedReservation' | extend sortCost = cost)
| order by sortCost asc
| project ReservationName, ChargeType, cost
| render barchart with (kind=stacked,title="Top 10 Most Underused Reservations: Accumulated Past 5 Bills")

In [ ]:
%%kql topids <<
let last5Bills = (Usage | distinct BillingPeriodStartDate | top 5 by BillingPeriodStartDate);
let top10res = Usage
| where BillingPeriodStartDate in (last5Bills)
| where ChargeType == "UnusedReservation"
| summarize UnusedCost=sum(Cost) by ReservationId
| top 10 by UnusedCost
| project ReservationId;
Usage
| where ReservationId in (top10res)
| where ChargeType == "UnusedReservation"
| summarize unused_cost=sum(Cost) by ReservationId, ReservationName
| top 10 by unused_cost

In [ ]:
top_ids_df = topids.to_dataframe()
reservations_df = reservations.reservations_dataframe()
reservations_df = reservations_df[reservations_df.provisioning_state == "Succeeded"]
if len(reservations_df[(reservations_df.applied_scope_type != 'Single') | (reservations_df.applied_scopes.map(lambda x: len(x)) != 1)]):
    raise 'Unsupported'
now = pd.Timestamp.utcnow()
reservations_df = reservations_df \
    .assign(subscription_id=reservations_df.applied_scopes.map(lambda x: resource_id.subscription_id(x[0]))) \
    .assign(term_used=(now - reservations_df.effective_date_time)/reservations_df.term) \
    .assign(reservation_id=reservations_df.name.str.split('/', expand=True)[1])

reservations_df = reservations_df[reservations_df.reservation_id.isin(top_ids_df.ReservationId)].set_index('reservation_id')

In [ ]:
top_res = reservations_df.join(topids.to_dataframe().set_index('ReservationId'), how='outer').sort_values('unused_cost', ascending=False)
top_res = top_res.assign(name=top_res.display_name.fillna(top_res.ReservationName)).drop(columns='ReservationName')
top_res[['name', 'location', 'sku', 'quantity', 'instance_flexibility', 'term', 'term_used', 'unused_cost']].assign(term=top_res.term/pd.Timedelta('365D')).style.hide_index().format('{}', na_rep='').format('${:,.2f}', subset='unused_cost').format('{:g}Y', na_rep='', subset='term').format('{:.0%}', na_rep='', subset='term_used').background_gradient(subset='unused_cost')